<a href="https://colab.research.google.com/github/subodh2708/deep-learning---tensorflow-2.0/blob/main/08_NLP_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP in Tensorflow

In [1]:
!nvidia-smi -L

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

from helper_functions import unzip_data,create_tensorboard_callback,plot_loss_curves,compare_historys

--2022-09-18 06:19:52--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-09-18 06:19:53 (73.1 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
unzip_data('nlp_getting_started.zip')


--2022-09-18 06:19:56--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.2.128, 2607:f8b0:4023:c0d::80
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.2.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2022-09-18 06:19:56 (144 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [4]:
 import pandas as pd
 train_df = pd.read_csv("train.csv")
 test_df = pd.read_csv("test.csv")
 train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
train_df_shuffled = train_df.sample(frac=1,random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [6]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [7]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [8]:
len(train_df),len(test_df)

(7613, 3263)

In [9]:
import random
random_index = random.randint(0,len(train_df)-5)
for row in train_df_shuffled[['text','target']][random_index:random_index+5].itertuples():
  _,text,target = row
  print(f"target:{target}","(real disaster)" if target > 0 else  "(not a disaster)")
  print(f"Text\n {text} \n")
  print("---\n")

target:0 (not a disaster)
Text
 I added a video to a @YouTube playlist http://t.co/O7QOgmOegU S@X 109 - SWS | Fatality (Captain Falcon) Vs. Snow (Fox) SSB4 Losers 

---

target:1 (real disaster)
Text
 @daisycuttertz @GROGParty @Tony_Burke He's been having lessons in suicide bombing from his imam! 

---

target:0 (not a disaster)
Text
 Me and all my friends could destroy this in 2hours lmao  https://t.co/waCtT18gdA 

---

target:0 (not a disaster)
Text
 My brains going to explode i need to leave this house. Ill be out smoking packs if you need me 

---

target:0 (not a disaster)
Text
 Don't like those head first slides. Especially into home !! #danger 

---



## splitting the data into training and validation sets

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
train_sentences, val_sentences,train_labels,val_labels = train_test_split(train_df_shuffled['text'].to_numpy(),
                                                                          train_df_shuffled['target'].to_numpy(),
                                                                          test_size = 0.1,
                                                                          random_state = 42)

## converting text into numbers

In [12]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

text_vectorizer = TextVectorization(max_tokens = None,
                                    standardize = "lower_and_strip_punctuation",
                                    split = 'whitespace',
                                    ngrams = None,
                                    output_mode = "int",
                                    output_sequence_length = None,
                                    pad_to_max_tokens= False
                                    )

In [13]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [14]:
max_vocab_length = 10000
max_length = 15

In [15]:
text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode = 'int',
                                    output_sequence_length = max_length)

In [16]:
text_vectorizer.adapt(train_sentences)

In [17]:
sample_sentence = "there is flood in my street"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 74,   9, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [18]:
random_sentence = random.choice(train_sentences)
print(f'Original text:\n {random_sentence} \
      \n Vectorized version:\n {text_vectorizer([random_sentence])}')

Original text:
 Experts in France begin examining airplane debris found on Reunion Island: French air accident experts on Wedn... http://t.co/v4SMAESLK5       
 Vectorized version:
 [[ 854    4 1021 1223 1413  444  247  225   11  432  419 1409  307  128
   854]]


In [19]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f"number of words in vocab:{len(words_in_vocab)}")
print(f"top 5 word in vocab:{top_5_words}")
print(f"bottom 5 words in vocab:{bottom_5_words}")

number of words in vocab:10000
top 5 word in vocab:['', '[UNK]', 'the', 'a', 'in']
bottom 5 words in vocab:['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


## Creating the Embedding using Embedding layer

In [20]:
from tensorflow.keras.layers import Embedding

embedding = Embedding(input_dim = max_vocab_length,
                      output_dim = 128,
                      embeddings_initializer ='uniform',
                      input_length = max_length)
embedding

In [21]:
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence} \
        \n Embedded Version:")

sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 AngelRiveraLibÛ_ #Snowden 'may have' broken laws? Hurricane Katrina may have caused some damage. http://t.co/jAaWuiOvdc Without Snowden hÛ_         
 Embedded Version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.04104841,  0.04448913, -0.0495089 , ...,  0.00157502,
         -0.03374593,  0.02687314],
        [-0.03911711,  0.04251145,  0.03060791, ...,  0.0454681 ,
         -0.01815307, -0.03665508],
        [ 0.0050079 ,  0.0003987 , -0.02832278, ...,  0.04500157,
          0.01183588, -0.02727689],
        ...,
        [ 0.00208094, -0.02417913, -0.01479955, ...,  0.00313465,
          0.02042843, -0.03292197],
        [-0.04104841,  0.04448913, -0.0495089 , ...,  0.00157502,
         -0.03374593,  0.02687314],
        [ 0.03897348, -0.030142  , -0.03971784, ...,  0.03884411,
         -0.01646965, -0.00735605]]], dtype=float32)>

In [22]:
random_sentence ,sample_embed[0][0],sample_embed[0][0].shape

("AngelRiveraLib\x89Û_ #Snowden 'may have' broken laws? Hurricane Katrina may have caused some damage. http://t.co/jAaWuiOvdc Without Snowden h\x89Û_",
 <tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-4.1048408e-02,  4.4489134e-02, -4.9508896e-02, -5.0367825e-03,
         2.0408034e-03,  2.4185147e-02, -5.4770000e-03,  3.0566994e-02,
        -3.1827115e-02, -3.4833990e-02,  1.9521389e-02,  2.6867140e-02,
         1.0802399e-02, -3.4437060e-02,  2.8044209e-03, -2.3723876e-02,
         2.4402764e-02,  4.5955781e-02,  3.8135875e-02, -2.2172416e-02,
         1.8557791e-02,  3.7675451e-02,  2.9372621e-02,  2.6739463e-03,
         8.7804906e-03, -5.2045099e-03, -1.8699311e-02,  4.7472481e-02,
        -4.7945488e-02, -4.8973013e-02,  3.6694597e-02, -2.9569471e-02,
        -1.6448319e-02, -2.8580964e-02, -1.1801161e-02, -3.2807160e-02,
        -4.6795785e-02,  8.3100051e-05,  2.4100278e-02, -2.0212306e-02,
         4.7082257e-02,  1.8192455e-04,  4.2326737e-02, -2.8317118e-02,
       

## Modelling text Dataset 

## Model 0: Getting the baseline model

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', MultinomialNB())
])

model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [24]:
baseline_score = model_0.score(val_sentences, val_labels)
print(f' baseline accuracy : {baseline_score *100:.2f}%')

 baseline accuracy : 79.27%


In [25]:
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

## creating our evaluation function for our model experiments

In [26]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_result(y_true,y_pred):
  model_accuracy = accuracy_score(y_true,y_pred)
  model_precision,model_recall,model_f1,_ = precision_recall_fscore_support(y_true,y_pred,average = 'weighted')
  model_results = {'accuracy':model_accuracy,
                   'precision':model_precision,
                   'recall':model_recall,
                   'f1':model_f1}

  return model_results


In [27]:
baseline_results = calculate_result(y_true=val_labels,
                                    y_pred = baseline_preds)
print(baseline_results)

{'accuracy': 0.7926509186351706, 'precision': 0.8111390004213173, 'recall': 0.7926509186351706, 'f1': 0.7862189758049549}


##Model 1: simple dense model

In [28]:
from helper_functions import create_tensorboard_callback

In [29]:
SAVE_DIR = 'model_logs'

In [30]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype= tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model_1 = tf.keras.Model(inputs,outputs, name= 'model_1_dense')

In [31]:
model_1.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [32]:
model_1_history= model_1.fit(x = train_sentences,
                           y= train_labels,
                           epochs = 5,
                           validation_data = (val_sentences,val_labels),
                           callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,
                                                                    experiment_name = 'model_1_dense')])

Saving TensorBoard log files to: model_logs/model_1_dense/20220918-061958
Epoch 1/5
215/215 [==============================] - 7s 28ms/step - loss: 0.6103 - accuracy: 0.6945 - val_loss: 0.5402 - val_accuracy: 0.7441
Epoch 2/5
215/215 [==============================] - 6s 27ms/step - loss: 0.4413 - accuracy: 0.8171 - val_loss: 0.4692 - val_accuracy: 0.7861
Epoch 3/5
215/215 [==============================] - 6s 26ms/step - loss: 0.3455 - accuracy: 0.8599 - val_loss: 0.4602 - val_accuracy: 0.7887
Epoch 4/5
215/215 [==============================] - 5s 25ms/step - loss: 0.2836 - accuracy: 0.8927 - val_loss: 0.4672 - val_accuracy: 0.7887
Epoch 5/5
215/215 [==============================] - 5s 25ms/step - loss: 0.2366 - accuracy: 0.9121 - val_loss: 0.4781 - val_accuracy: 0.7756


In [33]:
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 6ms/step - loss: 0.4781 - accuracy: 0.7756


[0.47809311747550964, 0.7755905389785767]

In [34]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[0:10]

array([[0.3672259 ],
       [0.6957977 ],
       [0.9974524 ],
       [0.13422549],
       [0.10263109],
       [0.9440831 ],
       [0.91468996],
       [0.9929743 ],
       [0.9621309 ],
       [0.27027243]], dtype=float32)

In [35]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [36]:
model_1_results = calculate_result(y_true = val_labels,
                                   y_pred = model_1_preds)
model_1_results

{'accuracy': 0.7755905511811023,
 'precision': 0.7786932114612464,
 'recall': 0.7755905511811023,
 'f1': 0.7729519699688302}

In [37]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))


array([False, False, False, False])

## Visualising learned embeddings

In [38]:
word_in_vocab = text_vectorizer.get_vocabulary()
len(word_in_vocab) , words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [39]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [40]:
embed_weights = model_1.get_layer("embedding").get_weights()[0]
embed_weights.shape

(10000, 128)

In [41]:
import io

# Code to save trained embeddings to file - we got this from here: https://www.tensorflow.org/tutorials/text/word_embeddings#retrieve_the_trained_word_embeddings_and_save_them_to_disk
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [42]:
# Let's download the saved embeddings locally
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Recurrent Neural Networks

###Model 2 : LSTM

In [43]:
# Create LSTM model
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
print(f"After embedding: {x.shape}")
# x = layers.LSTM(64, activation="tanh", return_sequences=True)(x) # use return_sequences=True if you want to stack recurrent layers 
# print(f"After LSTM cell with return_sequences=True: {x.shape}")
x = layers.LSTM(64, activation="tanh")(x)
print(f"After LSTM cell: {x.shape}")
# x = layers.Dense(64, activation="relu")(x) # optional dense layer to have on top of LSTM layer
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

After embedding: (None, 15, 128)
After LSTM cell: (None, 64)


In [44]:
model_2.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [45]:
history_2 = model_2.fit(train_sentences,
                        train_labels,
                        epochs= 5,
                        validation_data = (val_sentences, val_labels),
                        callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,
                                                                 experiment_name = "model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20220918-062033
Epoch 1/5
215/215 [==============================] - 19s 68ms/step - loss: 0.2127 - accuracy: 0.9298 - val_loss: 0.5300 - val_accuracy: 0.7822
Epoch 2/5
215/215 [==============================] - 6s 28ms/step - loss: 0.1561 - accuracy: 0.9410 - val_loss: 0.5927 - val_accuracy: 0.7848
Epoch 3/5
215/215 [==============================] - 6s 27ms/step - loss: 0.1256 - accuracy: 0.9523 - val_loss: 0.8095 - val_accuracy: 0.7900
Epoch 4/5
215/215 [==============================] - 6s 27ms/step - loss: 0.1028 - accuracy: 0.9604 - val_loss: 0.7790 - val_accuracy: 0.7782
Epoch 5/5
215/215 [==============================] - 6s 28ms/step - loss: 0.0806 - accuracy: 0.9658 - val_loss: 0.8850 - val_accuracy: 0.7808


In [46]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs.shape, model_2_pred_probs[:10]

((762, 1), array([[7.1166962e-02],
        [8.7999833e-01],
        [9.9967647e-01],
        [1.8612239e-01],
        [2.6166439e-04],
        [9.8908973e-01],
        [4.8519108e-01],
        [9.9968237e-01],
        [9.9959373e-01],
        [7.3772329e-01]], dtype=float32))

In [47]:
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 0., 1., 1., 1.], dtype=float32)>

In [48]:
model_2_results = calculate_result(y_true= val_labels,
                                   y_pred = model_2_preds)
model_2_results

{'accuracy': 0.7808398950131233,
 'precision': 0.7809067257781027,
 'recall': 0.7808398950131233,
 'f1': 0.7798778043941109}

In [49]:
baseline_results

{'accuracy': 0.7926509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 3: GRU

In [52]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype= 'string')
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64)(x)
x = layers.Dense(64, activation= 'relu')(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model_3 = tf.keras.Model(inputs,outputs, name='model_3_GRU')

In [53]:
model_3.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [55]:
history_3 = model_3.fit(train_sentences,
                        train_labels,
                        epochs =5,
                        validation_data =(val_sentences, val_labels),
                        callbacks= [create_tensorboard_callback(dir_name= SAVE_DIR,
                                                                experiment_name='model_3_GRU')])

Saving TensorBoard log files to: model_logs/model_3_GRU/20220918-063617
Epoch 1/5
215/215 [==============================] - 8s 29ms/step - loss: 0.1489 - accuracy: 0.9491 - val_loss: 0.8067 - val_accuracy: 0.7756
Epoch 2/5
215/215 [==============================] - 6s 26ms/step - loss: 0.0836 - accuracy: 0.9676 - val_loss: 0.8350 - val_accuracy: 0.7730
Epoch 3/5
215/215 [==============================] - 6s 26ms/step - loss: 0.0691 - accuracy: 0.9727 - val_loss: 1.1362 - val_accuracy: 0.7717
Epoch 4/5
215/215 [==============================] - 6s 26ms/step - loss: 0.0558 - accuracy: 0.9777 - val_loss: 1.3199 - val_accuracy: 0.7769
Epoch 5/5
215/215 [==============================] - 6s 26ms/step - loss: 0.0515 - accuracy: 0.9771 - val_loss: 1.2342 - val_accuracy: 0.7651


In [56]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

array([[1.1884570e-03],
       [7.9333019e-01],
       [9.9999762e-01],
       [2.0118946e-01],
       [5.1863681e-05],
       [9.9985635e-01],
       [7.2357839e-01],
       [9.9999738e-01],
       [9.9999088e-01],
       [5.6986320e-01]], dtype=float32)

In [57]:
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [61]:
model_3_results = calculate_result(y_true= val_labels,
                                   y_pred = model_3_preds)
model_3_results

{'accuracy': 0.7650918635170604,
 'precision': 0.7681519700378248,
 'recall': 0.7650918635170604,
 'f1': 0.7621795783524195}